In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import glob
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# ===============================
# 1. User Settings (Parameters and Options)
# ===============================
N = 250  # 분석할 데이터 기간 (최근 N일)
data_folder = '/content/drive/MyDrive/Data/FNC/Historical_Data/'

# -------------------------------
# [Long System 1] L1 and Pyramiding Options
# -------------------------------
use_L1 = True
use_L1_pyramid = True
L1a = 20       # 최근 L1a일 중 최고가 돌파시 진입
L1b = 10       # 진입 후 최근 L1b일 중 최저가 하향시 청산
L1c = 2.0      # Exit Rule 1: (L1c × ATR)
L1d = 20       # ATR 기간 (Exit Rule 1)
L1Pa = 0.5     # 피라미딩 추가 진입: 계수
L1Pb = 20      # 피라미딩 ATR 기간
L1Pc = 3       # 최대 피라미딩 횟수

# -------------------------------
# [Long System 2] L2 and Pyramiding Options
# -------------------------------
use_L2 = True
use_L2_pyramid = True
L2a = 55
L2b = 20
L2c = 2.0
L2d = 20
L2Pa = 0.5
L2Pb = 20
L2Pc = 3

# -------------------------------
# [Short System 1] S1 and Pyramiding Options
# -------------------------------
use_S1 = True
use_S1_pyramid = True
S1a = 20
S1b = 10
S1c = 2.0
S1d = 20
S1Pa = 0.5
S1Pb = 20
S1Pc = 3

# -------------------------------
# [Short System 2] S2 and Pyramiding Options
# -------------------------------
use_S2 = True
use_S2_pyramid = True
S2a = 55
S2b = 20
S2c = 2.0
S2d = 20
S2Pa = 0.5
S2Pb = 20
S2Pc = 3

# -------------------------------
# Exit Rule 선택 (1 또는 2)
exit_rule_long = 1
exit_rule_short = 1

# -------------------------------
# Run Backtest?
use_backtest = True

# ===============================
# 2. Helper Functions
# ===============================
def format_date(date_val):
    return pd.to_datetime(date_val).strftime('%Y-%m-%d')

def ATR(data, period):
    high = data['High']
    low = data['Low']
    close = data['Close']
    prev_close = close.shift(1)
    tr1 = high - low
    tr2 = (high - prev_close).abs()
    tr3 = (low - prev_close).abs()
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(window=period, min_periods=1).mean()
    return atr

# ===============================
# 3. Simulation Functions for Each System
# ===============================
def simulate_long_system(df, entry_lb, exit_lb, exit_mult, use_pyramid, pyramid_mult, max_pyramid):
    trades = []
    position = None
    last_entry_price = None
    last_entry_atr = None
    pyramid_count = 0

    for i in range(entry_lb, len(df)):
        row = df.iloc[i]
        date = row['Date']
        close_price = row['Close']
        if position is None:
            if close_price > df.iloc[i-entry_lb:i]['High'].max():
                position = {'entries': [(date, close_price)], 'entry_index': i}
                last_entry_price = close_price
                last_entry_atr = row['ATR_L1d']
                pyramid_count = 0
        else:
            if use_pyramid and pyramid_count < max_pyramid:
                threshold = last_entry_price + (pyramid_mult * row['ATR_L1Pb'])
                if close_price > threshold:
                    position['entries'].append((date, close_price))
                    last_entry_price = close_price
                    last_entry_atr = row['ATR_L1d']
                    pyramid_count += 1
            exit_flag = False
            if close_price < df.iloc[i-exit_lb:i]['Low'].min():
                exit_flag = True
            elif exit_rule_long == 1:
                threshold_exit = last_entry_price - (exit_mult * last_entry_atr)
                if close_price < threshold_exit:
                    exit_flag = True
            if exit_flag:
                entries = position['entries']
                first_entry_price = entries[0][1]
                profit_pct = (close_price / first_entry_price - 1) * 100
                trades.append({
                    'entries': entries,
                    'entry_date': entries[0][0],
                    'entry_index': position['entry_index'],
                    'entry_price': first_entry_price,
                    'exit_date': date,
                    'exit_index': i,
                    'exit_price': close_price,
                    'profit_pct': profit_pct,
                    'system': 'L1'
                })
                position = None
                last_entry_price = None
                last_entry_atr = None
                pyramid_count = 0

    open_trade = None
    if position is not None:
        entries = position['entries']
        first_entry_price = entries[0][1]
        exit_threshold = last_entry_price - (exit_mult * last_entry_atr) if exit_rule_long==1 else None
        open_trade = {
            'entries': entries,
            'entry_date': entries[0][0],
            'entry_index': position['entry_index'],
            'entry_price': first_entry_price,     # 최초 진입 가격 (참고용)
            'last_entry_price': last_entry_price,   # 피라미딩 반영한 마지막 진입 가격
            'current_date': df.iloc[-1]['Date'],
            'current_price': df.iloc[-1]['Close'],
            'profit_pct': (df.iloc[-1]['Close'] / first_entry_price - 1) * 100,
            'exit_threshold': exit_threshold,
            'system': 'L1'
        }
    return trades, open_trade

def simulate_long_system2(df, entry_lb, exit_lb, exit_mult, use_pyramid, pyramid_mult, max_pyramid):
    trades = []
    position = None
    last_entry_price = None
    last_entry_atr = None
    pyramid_count = 0

    for i in range(entry_lb, len(df)):
        row = df.iloc[i]
        date = row['Date']
        close_price = row['Close']
        if position is None:
            if close_price > df.iloc[i-entry_lb:i]['High'].max():
                position = {'entries': [(date, close_price)], 'entry_index': i}
                last_entry_price = close_price
                last_entry_atr = row['ATR_L2d']
                pyramid_count = 0
        else:
            if use_pyramid and pyramid_count < max_pyramid:
                threshold = last_entry_price + (pyramid_mult * row['ATR_L2Pb'])
                if close_price > threshold:
                    position['entries'].append((date, close_price))
                    last_entry_price = close_price
                    last_entry_atr = row['ATR_L2d']
                    pyramid_count += 1
            exit_flag = False
            if close_price < df.iloc[i-exit_lb:i]['Low'].min():
                exit_flag = True
            elif exit_rule_long == 1:
                threshold_exit = last_entry_price - (exit_mult * last_entry_atr)
                if close_price < threshold_exit:
                    exit_flag = True
            if exit_flag:
                entries = position['entries']
                first_entry_price = entries[0][1]
                profit_pct = (close_price / first_entry_price - 1) * 100
                trades.append({
                    'entries': entries,
                    'entry_date': entries[0][0],
                    'entry_index': position['entry_index'],
                    'entry_price': first_entry_price,
                    'exit_date': date,
                    'exit_index': i,
                    'exit_price': close_price,
                    'profit_pct': profit_pct,
                    'system': 'L2'
                })
                position = None
                last_entry_price = None
                last_entry_atr = None
                pyramid_count = 0

    open_trade = None
    if position is not None:
        entries = position['entries']
        first_entry_price = entries[0][1]
        exit_threshold = last_entry_price - (exit_mult * last_entry_atr) if exit_rule_long==1 else None
        open_trade = {
            'entries': entries,
            'entry_date': entries[0][0],
            'entry_index': position['entry_index'],
            'entry_price': first_entry_price,
            'last_entry_price': last_entry_price,
            'current_date': df.iloc[-1]['Date'],
            'current_price': df.iloc[-1]['Close'],
            'profit_pct': (df.iloc[-1]['Close'] / first_entry_price - 1) * 100,
            'exit_threshold': exit_threshold,
            'system': 'L2'
        }
    return trades, open_trade

def simulate_short_system(df, entry_lb, exit_lb, exit_mult, use_pyramid, pyramid_mult, max_pyramid):
    trades = []
    position = None
    last_entry_price = None
    last_entry_atr = None
    pyramid_count = 0

    for i in range(entry_lb, len(df)):
        row = df.iloc[i]
        date = row['Date']
        close_price = row['Close']
        if position is None:
            if close_price < df.iloc[i-entry_lb:i]['Low'].min():
                position = {'entries': [(date, close_price)], 'entry_index': i}
                last_entry_price = close_price
                last_entry_atr = row['ATR_S1d']
                pyramid_count = 0
        else:
            if use_pyramid and pyramid_count < max_pyramid:
                threshold = last_entry_price - (pyramid_mult * row['ATR_S1Pb'])
                if close_price < threshold:
                    position['entries'].append((date, close_price))
                    last_entry_price = close_price
                    last_entry_atr = row['ATR_S1d']
                    pyramid_count += 1
            exit_flag = False
            if close_price > df.iloc[i-exit_lb:i]['High'].max():
                exit_flag = True
            elif exit_rule_short == 1:
                threshold_exit = last_entry_price + (exit_mult * last_entry_atr)
                if close_price > threshold_exit:
                    exit_flag = True
            if exit_flag:
                entries = position['entries']
                first_entry_price = entries[0][1]
                profit_pct = (first_entry_price / close_price - 1) * 100
                trades.append({
                    'entries': entries,
                    'entry_date': entries[0][0],
                    'entry_index': position['entry_index'],
                    'entry_price': first_entry_price,
                    'exit_date': date,
                    'exit_index': i,
                    'exit_price': close_price,
                    'profit_pct': profit_pct,
                    'system': 'S1'
                })
                position = None
                last_entry_price = None
                last_entry_atr = None
                pyramid_count = 0

    open_trade = None
    if position is not None:
        entries = position['entries']
        first_entry_price = entries[0][1]
        exit_threshold = last_entry_price + (exit_mult * last_entry_atr) if exit_rule_short==1 else None
        open_trade = {
            'entries': entries,
            'entry_date': entries[0][0],
            'entry_index': position['entry_index'],
            'entry_price': first_entry_price,
            'last_entry_price': last_entry_price,
            'current_date': df.iloc[-1]['Date'],
            'current_price': df.iloc[-1]['Close'],
            'profit_pct': (first_entry_price / df.iloc[-1]['Close'] - 1) * 100,
            'exit_threshold': exit_threshold,
            'system': 'S1'
        }
    return trades, open_trade

def simulate_short_system2(df, entry_lb, exit_lb, exit_mult, use_pyramid, pyramid_mult, max_pyramid):
    trades = []
    position = None
    last_entry_price = None
    last_entry_atr = None
    pyramid_count = 0

    for i in range(entry_lb, len(df)):
        row = df.iloc[i]
        date = row['Date']
        close_price = row['Close']
        if position is None:
            if close_price < df.iloc[i-entry_lb:i]['Low'].min():
                position = {'entries': [(date, close_price)], 'entry_index': i}
                last_entry_price = close_price
                last_entry_atr = row['ATR_S2d']
                pyramid_count = 0
        else:
            if use_pyramid and pyramid_count < max_pyramid:
                threshold = last_entry_price - (pyramid_mult * row['ATR_S2Pb'])
                if close_price < threshold:
                    position['entries'].append((date, close_price))
                    last_entry_price = close_price
                    last_entry_atr = row['ATR_S2d']
                    pyramid_count += 1
            exit_flag = False
            if close_price > df.iloc[i-exit_lb:i]['High'].max():
                exit_flag = True
            elif exit_rule_short == 1:
                threshold_exit = last_entry_price + (exit_mult * last_entry_atr)
                if close_price > threshold_exit:
                    exit_flag = True
            if exit_flag:
                entries = position['entries']
                first_entry_price = entries[0][1]
                profit_pct = (first_entry_price / close_price - 1) * 100
                trades.append({
                    'entries': entries,
                    'entry_date': entries[0][0],
                    'entry_index': position['entry_index'],
                    'entry_price': first_entry_price,
                    'exit_date': date,
                    'exit_index': i,
                    'exit_price': close_price,
                    'profit_pct': profit_pct,
                    'system': 'S2'
                })
                position = None
                last_entry_price = None
                last_entry_atr = None
                pyramid_count = 0

    open_trade = None
    if position is not None:
        entries = position['entries']
        first_entry_price = entries[0][1]
        exit_threshold = last_entry_price + (exit_mult * last_entry_atr) if exit_rule_short==1 else None
        open_trade = {
            'entries': entries,
            'entry_date': entries[0][0],
            'entry_index': position['entry_index'],
            'entry_price': first_entry_price,
            'last_entry_price': last_entry_price,
            'current_date': df.iloc[-1]['Date'],
            'current_price': df.iloc[-1]['Close'],
            'profit_pct': (first_entry_price / df.iloc[-1]['Close'] - 1) * 100,
            'exit_threshold': exit_threshold,
            'system': 'S2'
        }
    return trades, open_trade

def calc_results(df, trades, system_type):
    cumulative_return = 1.0
    trade_mdds = []
    for trade in trades:
        first_entry = trade['entry_price']
        exit_price = trade['exit_price']
        if system_type.startswith('S'):
            trade_return = first_entry / exit_price - 1
        else:
            trade_return = exit_price / first_entry - 1
        cumulative_return *= (1 + trade_return)
        entry_idx = trade['entry_index']
        exit_idx = trade['exit_index']
        trade_slice = df.iloc[entry_idx: exit_idx+1]
        high_price = trade_slice['High'].max()
        low_price = trade_slice['Low'].min()
        if system_type.startswith('S'):
            trade_mdd = (high_price - low_price) / low_price * 100 if low_price != 0 else 0
        else:
            trade_mdd = (high_price - low_price) / high_price * 100 if high_price != 0 else 0
        trade_mdds.append(trade_mdd)
    cumulative_return_pct = (cumulative_return - 1) * 100
    overall_mdd = max(trade_mdds) if trade_mdds else 0
    return cumulative_return_pct, overall_mdd

# ===============================
# 4. Simulation Runner for a Single File
# ===============================
def run_simulation(file_path):
    output_lines = []
    # myprint: 내부 출력 내용을 저장만 하고, 콘솔에는 상세 출력은 하지 않음.
    def myprint(*args, **kwargs):
        s = " ".join(str(x) for x in args)
        output_lines.append(s)

    # 데이터 로드 및 전처리
    df = pd.read_csv(file_path)
    df.sort_values('Date', inplace=True)
    df = df.tail(N).reset_index(drop=True)

    # 각 시스템별 ATR 계산
    df['ATR_L1Pb'] = ATR(df, L1Pb)
    df['ATR_L1d'] = ATR(df, L1d)
    df['ATR_L2Pb'] = ATR(df, L2Pb)
    df['ATR_L2d'] = ATR(df, L2d)
    df['ATR_S1Pb'] = ATR(df, S1Pb)
    df['ATR_S1d'] = ATR(df, S1d)
    df['ATR_S2Pb'] = ATR(df, S2Pb)
    df['ATR_S2d'] = ATR(df, S2d)

    results = {}
    if use_L1:
        trades_L1, open_trade_L1 = simulate_long_system(df, L1a, L1b, L1c, use_L1_pyramid, L1Pa, L1Pc)
        results['L1'] = (trades_L1, open_trade_L1)
    if use_L2:
        trades_L2, open_trade_L2 = simulate_long_system2(df, L2a, L2b, L2c, use_L2_pyramid, L2Pa, L2Pc)
        results['L2'] = (trades_L2, open_trade_L2)
    if use_S1:
        trades_S1, open_trade_S1 = simulate_short_system(df, S1a, S1b, S1c, use_S1_pyramid, S1Pa, S1Pc)
        results['S1'] = (trades_S1, open_trade_S1)
    if use_S2:
        trades_S2, open_trade_S2 = simulate_short_system2(df, S2a, S2b, S2c, use_S2_pyramid, S2Pa, S2Pc)
        results['S2'] = (trades_S2, open_trade_S2)

    backtest_results = {}
    if use_backtest:
        for system, (trades, open_trade) in results.items():
            if trades:
                cum_ret, mdd = calc_results(df, trades, system)
                backtest_results[system] = {'cumulative_return_pct': cum_ret, 'MDD_pct': mdd}
            else:
                backtest_results[system] = {'cumulative_return_pct': None, 'MDD_pct': None}

    start_date = format_date(df.iloc[0]['Date'])
    end_date = format_date(df.iloc[-1]['Date'])
    myprint("===== Backtest Period =====")
    myprint("Start Date:", start_date, ", End Date:", end_date)

    myprint("\n===== Backtest Results =====")
    for system, result in backtest_results.items():
        if result['cumulative_return_pct'] is not None:
            myprint(f"{system}: Cumulative Return = {result['cumulative_return_pct']:.2f}%, MDD = {result['MDD_pct']:.2f}%")
        else:
            myprint(f"{system}: No Trades")

    myprint("\n===== Last Position Status by System =====")
    for system, (trades, open_trade) in results.items():
        if open_trade is not None:
            first_entry_date = format_date(open_trade['entry_date'])
            if system in ['L1', 'L2']:
                myprint(f"{system} Position Open - Entry Date: {first_entry_date}, Entry Price: {open_trade['entry_price']:.2f}, Current Price: {open_trade['current_price']:.2f}")
                if len(open_trade['entries']) > 1:
                    myprint("   Pyramiding Entry Details:")
                    for idx, (entry_date, entry_price) in enumerate(open_trade['entries']):
                        myprint(f"      Entry {idx+1}: Date: {format_date(entry_date)}, Price: {entry_price:.2f}")
                if exit_rule_long == 1 and open_trade['exit_threshold'] is not None:
                    pct_move = (open_trade['last_entry_price'] - open_trade['exit_threshold']) / open_trade['last_entry_price'] * 100
                    myprint(f"  >> Exit Rule Applied: Exit if price drops by {pct_move:.2f}% from last entry price")
            else:
                myprint(f"{system} Position Open - Entry Date: {first_entry_date}, Entry Price: {open_trade['entry_price']:.2f}, Current Price: {open_trade['current_price']:.2f}")
                if len(open_trade['entries']) > 1:
                    myprint("   Pyramiding Entry Details:")
                    for idx, (entry_date, entry_price) in enumerate(open_trade['entries']):
                        myprint(f"      Entry {idx+1}: Date: {format_date(entry_date)}, Price: {entry_price:.2f}")
                if exit_rule_short == 1 and open_trade['exit_threshold'] is not None:
                    pct_move = (open_trade['exit_threshold'] - open_trade['last_entry_price']) / open_trade['last_entry_price'] * 100
                    myprint(f"  >> Exit Rule Applied: Exit if price rises by {pct_move:.2f}% from last entry price")
        else:
            if trades:
                last_trade = trades[-1]
                myprint(f"{system} Last Trade - Entry Date: {format_date(last_trade['entry_date'])}, Entry Price: {last_trade['entry_price']:.2f}, Exit Date: {format_date(last_trade['exit_date'])}, Exit Price: {last_trade['exit_price']:.2f}, Return: {last_trade['profit_pct']:.2f}%")
                if len(last_trade['entries']) > 1:
                    myprint("   Pyramiding Entry Details:")
                    for idx, (entry_date, entry_price) in enumerate(last_trade['entries']):
                        myprint(f"      Entry {idx+1}: Date: {format_date(entry_date)}, Price: {entry_price:.2f}")
            else:
                myprint(f"{system}: No Trades")

    myprint("\n===== Final Trade Details and Pyramiding Entry Information by System =====")
    for system, (trades, open_trade) in results.items():
        if trades:
            for trade in trades:
                myprint(f"{system} Trade - Entry Date: {format_date(trade['entry_date'])}, Entry Price: {trade['entry_price']:.2f}, Exit Date: {format_date(trade['exit_date'])}, Exit Price: {trade['exit_price']:.2f}, Return: {trade['profit_pct']:.2f}%")
                if len(trade['entries']) > 1:
                    myprint("   Pyramiding Entry Details:")
                    for idx, (entry_date, entry_price) in enumerate(trade['entries']):
                        myprint(f"      Entry {idx+1}: Date: {format_date(entry_date)}, Price: {entry_price:.2f}")
        else:
            myprint(f"{system}: No Trades")

    last_day_str = format_date(df.iloc[-1]['Date'])
    myprint("\n===== Last Day Exit Signals ===== (1 if an exit signal occurred on the last day, else 0)")
    exit_signals = {}
    for system, (trades, open_trade) in results.items():
        flag_exit = 0
        for trade in trades:
            if format_date(trade['exit_date']) == last_day_str:
                flag_exit = 1
                break
        myprint(f"{system}: {flag_exit}")
        exit_signals[system] = flag_exit

    myprint("\n===== Last Day Open Signals ===== (1 if an entry signal occurred on the last day, else 0)")
    open_signals = {}
    for system, (trades, open_trade) in results.items():
        flag_open = 0
        for trade in trades:
            if format_date(trade['entry_date']) == last_day_str:
                flag_open = 1
                break
        if open_trade is not None:
            if format_date(open_trade['entries'][-1][0]) == last_day_str:
                flag_open = 1
        myprint(f"{system}: {flag_open}")
        open_signals[system] = flag_open

    myprint("\n===== Open Positions ===== (1 if a position is currently open, else 0)")
    open_positions = {}
    for system, (trades, open_trade) in results.items():
        flag_position = 1 if open_trade is not None else 0
        myprint(f"{system}: {flag_position}")
        open_positions[system] = flag_position

    backtest_summary = ""
    for system, res in backtest_results.items():
        if res['cumulative_return_pct'] is not None:
            backtest_summary += f"{system}: Return={res['cumulative_return_pct']:.2f}%, MDD={res['MDD_pct']:.2f}%; "
        else:
            backtest_summary += f"{system}: No Trades; "

    # --- 추가: L1, L2 시스템에 대한 진입 포지션의 청산 임계치 계산 ---
    thresholds = {}
    for sys in ['L1', 'L2']:
        trades_sys, open_trade = results.get(sys, ([], None))
        if open_trade is not None and open_trade.get("exit_threshold") is not None:
            # 진입가격 대비 청산 임계치 (%)
            pct_threshold = (open_trade["last_entry_price"] - open_trade["exit_threshold"]) / open_trade["last_entry_price"] * 100
            thresholds[sys] = round(pct_threshold, 2)
        else:
            thresholds[sys] = "no entry"

    sim_summary = {
        "backtest": backtest_summary,
        "exit_signals": exit_signals,
        "open_signals": open_signals,
        "open_positions": open_positions,
        "thresholds": thresholds
    }
    return "\n".join(output_lines), sim_summary

# ===============================
# 5. Process All Files and Save Results
# ===============================
text_results = []
summary_rows_exit = []
summary_rows_open = []
summary_rows_position = []

file_list = glob.glob(os.path.join(data_folder, "*.csv"))
total_files = len(file_list)

for idx, file in enumerate(file_list):
    symbol = os.path.splitext(os.path.basename(file))[0]
    sim_output, sim_summary = run_simulation(file)

    # 텍스트 파일용 결과 (헤더 포함)
    text_results.append("============")
    text_results.append(f"[{symbol}] {symbol}")
    text_results.append("")
    text_results.append(sim_output)
    text_results.append("============\n")

    # 각 signal 별로 1인 경우 summary row 추가 (여기서 L1_threshold, L2_threshold 추가)
    row_exit = {
        "Symbol": symbol,
        "Backtest Results": sim_summary["backtest"],
        "L1": sim_summary["exit_signals"].get("L1", 0),
        "L2": sim_summary["exit_signals"].get("L2", 0),
        "S1": sim_summary["exit_signals"].get("S1", 0),
        "S2": sim_summary["exit_signals"].get("S2", 0),
        "L1_threshold": sim_summary["thresholds"].get("L1", "no entry"),
        "L2_threshold": sim_summary["thresholds"].get("L2", "no entry")
    }
    if any(v == 1 for v in row_exit.values() if isinstance(v, int)):
        summary_rows_exit.append(row_exit)

    row_open = {
        "Symbol": symbol,
        "Backtest Results": sim_summary["backtest"],
        "L1": sim_summary["open_signals"].get("L1", 0),
        "L2": sim_summary["open_signals"].get("L2", 0),
        "S1": sim_summary["open_signals"].get("S1", 0),
        "S2": sim_summary["open_signals"].get("S2", 0),
        "L1_threshold": sim_summary["thresholds"].get("L1", "no entry"),
        "L2_threshold": sim_summary["thresholds"].get("L2", "no entry")
    }
    if any(v == 1 for v in row_open.values() if isinstance(v, int)):
        summary_rows_open.append(row_open)

    row_pos = {
        "Symbol": symbol,
        "Backtest Results": sim_summary["backtest"],
        "L1": sim_summary["open_positions"].get("L1", 0),
        "L2": sim_summary["open_positions"].get("L2", 0),
        "S1": sim_summary["open_positions"].get("S1", 0),
        "S2": sim_summary["open_positions"].get("S2", 0),
        "L1_threshold": sim_summary["thresholds"].get("L1", "no entry"),
        "L2_threshold": sim_summary["thresholds"].get("L2", "no entry")
    }
    if any(v == 1 for v in row_pos.values() if isinstance(v, int)):
        summary_rows_position.append(row_pos)

    # 진행률만 콘솔 출력
    print(f"Processing {idx+1}/{total_files}: {symbol}")

# 결과 텍스트 파일 저장 (파일명: TurtleResult_YYYYMMDD_HHMMSS.txt)
now_str = datetime.now().strftime("%Y%m%d_%H%M%S")
result_filename = f"/content/drive/MyDrive/Data/FNC/TT_Result_{now_str}.txt"
with open(result_filename, "w") as f:
    f.write("\n".join(text_results))
print(f"\n전체 종목 시뮬레이션 결과가 {result_filename} 파일에 저장되었습니다.")

# ===============================
# 6. Create Summary CSV File with Parsed Backtest Results
# ===============================
df_exit = pd.DataFrame(summary_rows_exit)
df_open = pd.DataFrame(summary_rows_open)
df_position = pd.DataFrame(summary_rows_position)

if not df_exit.empty:
    df_exit["Signal Type"] = "Last Day Exit Signals"
if not df_open.empty:
    df_open["Signal Type"] = "Last Day Open Signals"
if not df_position.empty:
    df_position["Signal Type"] = "Open Positions"

# 원래 컬럼: Signal Type, Symbol, Backtest Results, L1, L2, S1, S2, L1_threshold, L2_threshold
df_summary = pd.concat([df_exit, df_open, df_position], ignore_index=True)
df_summary = df_summary[["Signal Type", "Symbol", "Backtest Results", "L1", "L2", "S1", "S2", "L1_threshold", "L2_threshold"]]

# --------------------------------------------
# Backtest Results 문자열을 8개 컬럼으로 분리
# (L1_Return, L1_MDD, L2_Return, L2_MDD, S1_Return, S1_MDD, S2_Return, S2_MDD)
def parse_backtest_results(s):
    results = {
         "L1_Return": None, "L1_MDD": None,
         "L2_Return": None, "L2_MDD": None,
         "S1_Return": None, "S1_MDD": None,
         "S2_Return": None, "S2_MDD": None
    }
    if not isinstance(s, str):
        return results
    parts = s.split(";")
    for part in parts:
        part = part.strip()
        if not part:
            continue
        if part.startswith("L1:"):
            if "No Trades" in part:
                results["L1_Return"] = "No Trades"
                results["L1_MDD"] = "No Trades"
            else:
                match_ret = re.search(r"Return=([-\d\.]+)%", part)
                match_mdd = re.search(r"MDD=([-\d\.]+)%", part)
                if match_ret:
                    results["L1_Return"] = float(match_ret.group(1))
                if match_mdd:
                    results["L1_MDD"] = float(match_mdd.group(1))
        elif part.startswith("L2:"):
            if "No Trades" in part:
                results["L2_Return"] = "No Trades"
                results["L2_MDD"] = "No Trades"
            else:
                match_ret = re.search(r"Return=([-\d\.]+)%", part)
                match_mdd = re.search(r"MDD=([-\d\.]+)%", part)
                if match_ret:
                    results["L2_Return"] = float(match_ret.group(1))
                if match_mdd:
                    results["L2_MDD"] = float(match_mdd.group(1))
        elif part.startswith("S1:"):
            if "No Trades" in part:
                results["S1_Return"] = "No Trades"
                results["S1_MDD"] = "No Trades"
            else:
                match_ret = re.search(r"Return=([-\d\.]+)%", part)
                match_mdd = re.search(r"MDD=([-\d\.]+)%", part)
                if match_ret:
                    results["S1_Return"] = float(match_ret.group(1))
                if match_mdd:
                    results["S1_MDD"] = float(match_mdd.group(1))
        elif part.startswith("S2:"):
            if "No Trades" in part:
                results["S2_Return"] = "No Trades"
                results["S2_MDD"] = "No Trades"
            else:
                match_ret = re.search(r"Return=([-\d\.]+)%", part)
                match_mdd = re.search(r"MDD=([-\d\.]+)%", part)
                if match_ret:
                    results["S2_Return"] = float(match_ret.group(1))
                if match_mdd:
                    results["S2_MDD"] = float(match_mdd.group(1))
    return results

parsed = df_summary["Backtest Results"].apply(parse_backtest_results)
df_parsed = pd.DataFrame(list(parsed))
df_summary = pd.concat([df_summary.drop(columns=["Backtest Results"]), df_parsed], axis=1)

# 최종 컬럼 순서: Signal Type, Symbol, L1_Return, L1_MDD, L2_Return, L2_MDD, S1_Return, S1_MDD, S2_Return, S2_MDD, L1_threshold, L2_threshold, L1, L2, S1, S2
final_columns = ["Signal Type", "Symbol",
                 "L1_Return", "L1_MDD", "L2_Return", "L2_MDD",
                 "S1_Return", "S1_MDD", "S2_Return", "S2_MDD",
                 "L1_threshold", "L2_threshold",
                 "L1", "L2", "S1", "S2"]
df_summary = df_summary[final_columns]

summary_filename = f"/content/drive/MyDrive/Data/FNC/TT_Summary_{now_str}.csv"
df_summary.to_csv(summary_filename, index=False)
print(f"요약 정보가 {summary_filename} 파일에 저장되었습니다.")

# -----------------------------------------------------------
# 추가: Last Day Open Signals의 L1 또는 L2에 1이 있는 종목 필터링
# -----------------------------------------------------------
df_open_filtered = df_summary[(df_summary["Signal Type"]=="Last Day Open Signals") & ((df_summary["L1"]==1) | (df_summary["L2"]==1))]

# Tickers 파일: 오직 티커(Symbol)만 저장 (헤더는 "[Symbol]")
tickers_filepath = f"/content/drive/MyDrive/Data/FNC/TT_Long_List_{now_str}.csv"
df_tickers = df_open_filtered[["Symbol"]]
df_tickers.to_csv(tickers_filepath, index=False, header=["[Symbol]"])
df_tickers.to_csv("/content/drive/MyDrive/Data/FNC/TT_Long_List.csv", index=False, header=["[Symbol]"])
print(f"Tickers 파일이 {tickers_filepath} 에 저장되었습니다.")

# Tickers Summary 파일: Symbol, L1_Return, L1_MDD, L2_Return, L2_MDD, L1_threshold, L2_threshold 컬럼만 저장
tickers_summary_filepath = f"/content/drive/MyDrive/Data/FNC/TT_Long_List_Summary_{now_str}.csv"
df_tickers_summary = df_open_filtered[["Symbol", "L1_Return", "L1_MDD", "L2_Return", "L2_MDD", "L1_threshold", "L2_threshold"]]
df_tickers_summary.to_csv(tickers_summary_filepath, index=False)
print(f"Tickers Summary 파일이 {tickers_summary_filepath} 에 저장되었습니다.")


Processing 1/582: BTC-KRW
Processing 2/582: BTC-USD
Processing 3/582: XRP-KRW
Processing 4/582: XRP-USD
Processing 5/582: PLTR
Processing 6/582: PLTU
Processing 7/582: PLTD
Processing 8/582: IBM
Processing 9/582: INTC
Processing 10/582: AAPL
Processing 11/582: AAPD
Processing 12/582: NVDA
Processing 13/582: NVDD
Processing 14/582: MSFT
Processing 15/582: MSFD
Processing 16/582: GOOG
Processing 17/582: GGLS
Processing 18/582: AMZN
Processing 19/582: AMZD
Processing 20/582: META
Processing 21/582: METD


<ipython-input-6-75c409861f84>:304: RuntimeWarning: invalid value encountered in scalar divide
  'profit_pct': (first_entry_price / df.iloc[-1]['Close'] - 1) * 100,
<ipython-input-6-75c409861f84>:375: RuntimeWarning: invalid value encountered in scalar divide
  'profit_pct': (first_entry_price / df.iloc[-1]['Close'] - 1) * 100,
<ipython-input-6-75c409861f84>:486: RuntimeWarning: divide by zero encountered in scalar divide
  pct_move = (open_trade['exit_threshold'] - open_trade['last_entry_price']) / open_trade['last_entry_price'] * 100


Processing 22/582: TSLA
Processing 23/582: TSLS
Processing 24/582: SPY
Processing 25/582: DIA
Processing 26/582: QQQ
Processing 27/582: IWM
Processing 28/582: SH
Processing 29/582: DOG
Processing 30/582: PSQ
Processing 31/582: RWM
Processing 32/582: 069500.KS
Processing 33/582: 229200.KS
Processing 34/582: 114800.KS
Processing 35/582: 251340.KS
Processing 36/582: TLT
Processing 37/582: IEF
Processing 38/582: AGG
Processing 39/582: LQD
Processing 40/582: HYG
Processing 41/582: TBF
Processing 42/582: TBX
Processing 43/582: SJB
Processing 44/582: 152380.KS
Processing 45/582: 114460.KS
Processing 46/582: 272560.KS
Processing 47/582: 273130.KS
Processing 48/582: 217770.KS
Processing 49/582: 176950.KS
Processing 50/582: BITO
Processing 51/582: BITI
Processing 52/582: GLD
Processing 53/582: SLV
Processing 54/582: GLL
Processing 55/582: ZSL
Processing 56/582: GNR
Processing 57/582: XLE
Processing 58/582: XME
Processing 59/582: PPLT
Processing 60/582: DBB
Processing 61/582: CPER
Processing 62/5